# XGBOOST - RANDOM SEARCH

In [1]:
import numpy as np
from scipy.stats import uniform as sp_rand
from sklearn import datasets
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
import pandas as pd
from datetime import datetime
from time import time

### GLOBAL VARIABLES

In [2]:
DATAPATH = 'data/train_test/'
SEED = 47
NITER = 100
CV = 3
SCORE = 'roc_auc'
handlingnull = False
NJOBS = 7
USEGPU = False

### LOAD DATASET

In [3]:
train_features = pd.read_pickle(DATAPATH+'X_train.pkl')

In [8]:
train_labels = pd.read_pickle(DATAPATH+'y_train.pkl')['target']

In [6]:
train_features.shape

(148865, 1770)

In [12]:
### create a DMatrix and handling Null values
if handlingnull:
    #train_features[np.isnan(train_features)] = -9999
    xgtrain = xgb.DMatrix(train_features, train_labels, missing=-9999)
else:
    xgtrain = xgb.DMatrix(train_features.values, train_labels.values)

### TRAIN MODEL

#### Set hyperparameters

In [13]:
# ======== General Parameters ======= #

# Select the type of model to run at each iteration. gbtree or gblinear.
booster = 'gbtree'


# ======== Booster Parameters ======== # 

# Analogous to learning rate in GBM. 
# Typical final values to be used: 0.01-0.2
eta = [0.01] 

# Defines the minimum sum of weights of all observations required in a child.
min_child_weight = [i for i in range(1,10,2)]

# The maximum depth of a tree
max_depth = [i for i in range(3,10,2)] 

# A node is split only when the resulting split gives a positive reduction in the loss function. 
# Gamma specifies the minimum loss reduction required to make a split.
gamma = [i/10.0 for i in range(0,5)]

# Denotes the fraction of observations to be randomly samples for each tree.
subsample = [i/10.0 for i in range(6,10)]

# Denotes the fraction of columns to be randomly samples for each tree.
colsample_bytree = [i/10.0 for i in range(6,10)]

# L2 regularization term on weights (analogous to Ridge regression)
reg_lambda = [i/10.0 for i in range(4,10)]

# L1 regularization term on weight (analogous to Lasso regression)
reg_alpha = [0, 0.001, 0.005, 0.01, 0.05]

# Control the balance of positive and negative weights, useful for unbalanced classes. 
# A typical value to consider: sum(negative instances) / sum(positive instances)scale_pos_weight = 1
scale_pos_weight = int((len(train_labels) - np.sum(train_labels.values))/np.sum(train_labels.values))


# Learning Task Parameters

# This defines the loss function to be minimized. 
# - binary:logistic –logistic regression for binary classification, returns predicted probability (not class)
# - multi:softmax –multiclass classification using the softmax objective, returns predicted class (not probabilities)
#   you also need to set an additional num_class (number of classes) parameter defining the number of unique classes
# - multi:softprob –same as softmax, but returns predicted probability of each data point belonging to each class.
objective  = 'binary:logistic'


# The metric to be used for validation data.
# - rmse – root mean square error
# - mae – mean absolute error
# - logloss – negative log-likelihood
# - error – Binary classification error rate (0.5 threshold)
# - merror – Multiclass classification error rate
# - mlogloss – Multiclass logloss
# - auc: Area under the curve
eval_metric = 'auc'

[xgboost params](https://xgboost.readthedocs.io/en/latest/python/python_api.html)

In [14]:
# Create the random grid
random_grid = {
    'learning_rate' : eta,
    'min_child_weight' : min_child_weight,
    'max_depth' : max_depth,
    'gamma': gamma,
    'subsample': subsample,
    'colsample_bytree' : colsample_bytree,
    'reg_lambda' : reg_lambda,
    'reg_alpha' : reg_alpha,
}

In [15]:
random_grid

{'learning_rate': [0.01],
 'min_child_weight': [1, 3, 5, 7, 9],
 'max_depth': [3, 5, 7, 9],
 'gamma': [0.0, 0.1, 0.2, 0.3, 0.4],
 'subsample': [0.6, 0.7, 0.8, 0.9],
 'colsample_bytree': [0.6, 0.7, 0.8, 0.9],
 'reg_lambda': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'reg_alpha': [0, 0.001, 0.005, 0.01, 0.05]}

#### Find num boost

In [16]:
early_stopping_rounds = 50

In [17]:
model=XGBClassifier(seed=SEED, booster=booster, objective=objective,  scale_pos_weight = scale_pos_weight, nthread=NJOBS)
xgb_param = model.get_xgb_params()
xgb_param['objective'] = objective

if USEGPU:
    xgb_param['tree_method'] = 'gpu_hist'
    xgb_param['gpu_id'] = 0


In [18]:
cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round = 1000, nfold = CV, metrics = eval_metric, early_stopping_rounds = early_stopping_rounds, seed = SEED)

In [19]:
n_estimators = cvresult.shape[0]

In [20]:
print("Best number of boosters: ", n_estimators)

Best number of boosters:  313


In [22]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
model = XGBClassifier(n_estimators=n_estimators, scale_pos_weight=scale_pos_weight,  objective=objective)

In [23]:
if USEGPU:
    model.set_params(gpu_id = 0)
    model.set_params(tree_method='gpu_hist')

In [24]:
# Random search of parameters, using CV fold cross validation, 
# search across NITER different combinations, and use all available cores
xgboost_rsearch = RandomizedSearchCV(estimator = model, param_distributions = random_grid, scoring=SCORE, n_iter = NITER, cv = CV, verbose=2, random_state=SEED, n_jobs = NJOBS)# Fit the random search model


#### Training

In [ ]:
start = time()
xgboost_rsearch.fit(train_features, train_labels)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), NITER))

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  27 tasks      | elapsed: 135.2min
[Parallel(n_jobs=7)]: Done 148 tasks      | elapsed: 608.9min


#### Saving results

In [ ]:
cv_results = pd.DataFrame(xgboost_rsearch.cv_results_)

In [60]:
cv_results.to_csv('output/results/rsearch_xgboost_classifier_d' + str(datetime.now().date()) + '.csv',sep=';',index=False)

#### Best estimator

In [61]:
xgboost_rsearch.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.6, gamma=0.4, learning_rate=0.01,
       max_delta_step=0, max_depth=9, min_child_weight=9, missing=None,
       n_estimators=363, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0.05,
       reg_lambda=0.7, scale_pos_weight=9, seed=None, silent=True,
       subsample=0.7)

#### Best parameter

In [62]:
xgboost_rsearch.best_params_

{'subsample': 0.7,
 'reg_lambda': 0.7,
 'reg_alpha': 0.05,
 'min_child_weight': 9,
 'max_depth': 9,
 'learning_rate': 0.01,
 'gamma': 0.4,
 'colsample_bytree': 0.6}

#### Best Score

In [63]:
print(SCORE,' : ', xgboost_rsearch.best_score_)

roc_auc  :  0.7585701169823176


#### Saving best hyperparameters

In [64]:
np.save('output/hyperparameters/rseach_xgboost_classifier_bestparams_d' + str(datetime.now().date()) + '.npy', xgboost_rsearch.best_params_)

In [65]:
np.save('output/results/rseach_xgboost_classifier_best_estimator_d' + str(datetime.now().date()) + '.npy', xgboost_rsearch.best_estimator_)